# Custom Loss Function

In [2]:
# Imports

import torch 
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

In [3]:
# Define data
x = [
    [1,2],
    [3,4],
    [5,6],
    [7,8]
    ]

y = [
    [3],
    [7],
    [11],
    [15]
]

# Convert to tensor
X = torch.tensor(x).float()
Y = torch.tensor(y).float()

# Move to GPU
device = 'cuda' if torch.cuda.is_available() else 'cpu'

X = X.to(device)
Y = Y.to(device)

In [ ]:
# Prepare Dataset
class MyDataset(Dataset):
    def __init__(self, x, y):
        self.x = torch.tensor(x).float()
        self.y = torch.tensor(y).float()

    def __len__(self):
        return len(self.x)

    def __getitem__(self, index):
        return self.x[index], self.y[index]

# Initialize Dataset
ds = MyDataset(X,Y)

# DataLoader
dl = DataLoader(ds, batch_size=2, shuffle=True)


In [17]:
# Model Architecture

class MyNeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.input_to_hidden_layer = nn.Linear(2, 8)
        self.hidden_layer_activation = nn.ReLU()
        self.hidden_to_output_layer = nn.Linear(8, 1)

    def forward(self, x):
        x = self.input_to_hidden_layer(x)
        x = self.hidden_layer_activation(x)
        x = self.hidden_to_output_layer(x)
        return x

# Initialize model
mynet = MyNeuralNetwork().to(device)

In [15]:
# Original Loss Function
loss_fn = nn.MSELoss()

# Custom Loss Function
def mean_squared_error(_y, y):
    loss = (_y - y) ** 2
    loss = loss.mean()
    return loss 



# Optimizer
optimizer = torch.optim.SGD(mynet.parameters(), lr = 0.001)

In [18]:
# Training loop

loss_history = []

for _ in range(50):
    for data in dl:
        x, y = data 
        optimizer.zero_grad()
        loss_value = mean_squared_error(mynet(x), y)
        loss_value.backward()
        optimizer.step()
        loss_history.append(loss_value)

In [22]:
# Validation
val_x = [[10, 10]]

print("Original Loss: ", loss_fn(mynet(X), Y))
print("Custom Loss: ", mean_squared_error(mynet(X), Y))

Original Loss:  tensor(123.3615, device='cuda:0', grad_fn=<MseLossBackward>)
Custom Loss:  tensor(123.3615, device='cuda:0', grad_fn=<MeanBackward0>)
